In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as pil
import skimage as im
import astropy.io.fits as fits
from skimage.filters import threshold_otsu
from skimage.morphology import watershed
from skimage.morphology import binary_erosion as erode
from skimage.morphology import binary_dilation as dilate
from skimage.morphology import disk
from skimage.morphology import label
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
import matplotlib.colors as colors

%matplotlib inline
#%matplotlib widget

In [ ]:
def loadStack(mask, first, last, step=1) :
    img = np.array([1.0])
    indices=np.arange(first,last+1,step)
    for idx,filenum in enumerate(indices) :
        fname=mask.format(filenum)
        hdul = fits.open(fname)
        tmp = hdul[0].data.astype(float)
        
        if (idx==0):
            img.resize(tmp.shape[0],tmp.shape[1],len(indices))
            
        img[:,:,idx]=tmp
        
    return img

In [ ]:
dc=loadStack('/Users/data/P20180255/00_XCalibration/dc_{:05}.fits',1,10).astype(float).mean(axis=2);
ob=loadStack('/Users/data/P20180255/00_XCalibration/ob_{:05}.fits',1,10).astype(float).mean(axis=2);

In [ ]:
ob1=ob-dc
np.nan_to_num(ob1)
ob1[ob1<=0]=1
print(dc.min(),dc.max(),dc.mean())
print(ob1.min(),ob1.max(),ob1.mean())

In [ ]:
plt.figure(figsize=[12,5])
plt.subplot(1,2,1)
plt.imshow(dc,vmax=10000,vmin=0)
plt.title('DC')
plt.colorbar()
plt.subplot(1,2,2)
ob2=ob1[450:500,975:1050]
plt.imshow(ob1,vmax=10000,vmin=0)
plt.title('OB')
plt.colorbar()

In [ ]:
cal=loadStack('/Users/data/P20180255/00_XCalibration/xcal_{:05}.fits',1,361,10).astype(float)
print(cal.min(),cal.max(),cal.mean())
plt.imshow(cal[:,:,1],vmin=0.0,vmax=10000)

In [ ]:
def normalizeData(img,ob,dc) :
    for idx in np.arange(0, cal.shape[2]):
        tmp=(img[:,:,idx]-dc)
        tmp[tmp<=0]=1
        img[:,:,idx]=(tmp/ob1)
    lcal=-np.log(img)
    
    return lcal

In [ ]:
lcal=normalizeData(cal,ob,dc)

In [ ]:
idx=10
plt.figure(figsize=[12,5])
plt.subplot(1,2,1)
plt.imshow(lcal[:,:,idx],vmin=0,vmax=6)
plt.colorbar()
plt.subplot(1,2,2)
plt.plot(lcal[:,:,idx].mean(axis=0))


In [ ]:
def removeBaseline(img) :
    baseline = lcal.mean(axis=2).mean(axis=0)
    baseline=baseline.reshape(1,baseline.shape[0])

    b2=np.matmul(np.ones([lcal.shape[0],1]),baseline)
    res=img;
    for idx in np.arange(0,img.shape[2]) :
        res[:,:,idx]=res[:,:,idx]-b2
    return res

In [ ]:
bcal=removeBaseline(lcal)

In [ ]:
idx=5
m=bcal.mean()
s=bcal.std()
plt.figure(figsize=[12,5])
plt.subplot(1,2,1)
plt.imshow(m+1.96*s < bcal[:,:,idx],vmin=0,vmax=6)
plt.colorbar()
plt.subplot(1,2,2)
plt.plot(bcal[:,:,idx].mean(axis=0))

In [ ]:
def thresholdBBs(img,k) :
    s=bcal.std()
    m=bcal.mean()
    
    return (m+k*s)< img


In [ ]:
tcal=thresholdBBs(bcal,1.96)

In [ ]:
idx=10
fig,ax = plt.subplots(2,2,figsize=(15,5))
ax=ax.ravel()
ax[0].imshow(tcal[:,:,idx],vmin=0,vmax=2)
ax[1].plot(tcal[:,:,idx].mean(axis=0))
ecal = erode(tcal[:,:,idx],disk(12))
ax[2].imshow(ecal)
ax[3].plot(ecal.mean(axis=1))

In [ ]:
def findBeadsWS(img) :
    distance = ndi.distance_transform_edt(img)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((9,9)),labels=img)
    markers = ndi.label(local_maxi)[0]

    labels = watershed(-distance, markers, mask=img)
    h,ax=np.histogram(labels,bins=np.arange(0,labels.max()+1))
    m=h[np.argwhere(h<0.05*np.prod(labels.shape))].mean()
    s=h[np.argwhere(h<0.05*np.prod(labels.shape))].std()
    cog=[]
    for i in np.arange(0, h.size) :
        if ((m-s)<h[i]) and (h[i]<(m+s)) :
            selection=np.argwhere(labels==i)
            cog.append(selection.mean(axis=0))
       
    cog=np.asarray(cog)
    return cog

In [ ]:
def findBeads(img, selem, area = 3000, margin= 1500) :
    eimg = erode(img,selem)
     
    labels = label(eimg)
    cog=[]
    for i in np.arange(0, labels.max()) :
        selection=np.argwhere(labels==i)
        print(len(selection))
        if ((len(selection)<area+margin) and (area-margin<len(selection))) :
            cog.append(selection.mean(axis=0))
       
    cog=np.asarray(cog)
    return cog

In [ ]:
idx=10
cog=findBeads(tcal[:,:,idx],disk(12),area=3000,margin=1500)

In [ ]:
cog.shape

In [ ]:
plt.figure(figsize=[12,8])
plt.imshow(tcal[:,:,idx])

plt.plot(cog[:,1]-1,cog[:,0]-1,'r+')

In [ ]:
beadlist = []
bl2 = np.array([])
for idx in np.arange(0, tcal.shape[2]) :
    cog=findBeads(tcal[:,:,idx],disk(12),3000)
    beadlist.append(cog)
    bl2=np.append(bl2,cog)
    

In [ ]:
bl3=bl2
bl3=bl3.reshape(int(bl2.shape[0]/2),2 )

In [ ]:
plt.figure(figsize=[12,12])
plt.plot(bl3[:,0],bl3[:,1],'.')

In [ ]:
for bb in beadlist :
 #   print(bb.shape)
    plt.plot(bb[:,0],bb[:,1],'.')

In [ ]:
beadlist[0]

In [ ]:
cog = beadlist[0]
cog

In [ ]:
def rearrangeCOG(coglist,N) :

    res=coglist[0][0:N,:]
    
    for idx in np.arange(0,len(coglist)-1) :
        res=np.append(res,coglist[idx][0:N,:],axis=0)
        
    return res


In [ ]:
r=rearrangeCOG(beadlist,16)
print(r.shape,len(beadlist),r.shape[0]/len(beadlist))


# Estimate ellipse

$\dfrac {((X-C_x)\cos(\theta)+(Y-C_y)\sin(\theta))^2}{(R_x)^2}+\dfrac{((X-C_x) \sin(\theta)-(Y-C_y) \cos(\theta))^2}{(R_y)^2}=1$

There:
- (𝐶𝑥,𝐶𝑦) is the center of the Ellipse.
- 𝑅𝑥 is the Major-Radius, and 𝑅𝑦 is the Minor-Radius.
- 𝜃 is the angle of the Ellipse rotation.

In [ ]:
from skimage.measure import EllipseModel 

## Testing skimage ellipse

In [ ]:
xy = EllipseModel().predict_xy(np.linspace(0, 2 * np.pi, 25),params=(10, 15, 4, 8, np.deg2rad(30)))
ellipse = EllipseModel()
ellipse.estimate(xy)
print(ellipse.params, np.rad2deg(0.5235987755993301))
plt.plot(xy[:,0],xy[:,1])

# Old stuff

In [ ]:
mcal=np.median(cal[1:N:10],axis=0)

In [ ]:
d0=np.mean(cal[0,10:20,10:20])
for i in np.arange(1,N) :
    di=np.mean(cal[i,10:20,10:20]) 
    cal[i]=d0/di*cal[i]

In [ ]:
plt.figure(figsize=[15,8])
plt.subplot(1,2,1)
plt.imshow(mcal)
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(cal[1].reshape(mcal.shape)-mcal)
plt.colorbar()

In [ ]:
def minmax(img) :
    print(img.min(),img.max(), img.mean(), img.std())

In [ ]:
a=cal[4].reshape(mcal.shape)-mcal
a=a[:,350:1400]
plt.subplot(1,2,1)
plt.imshow(a)
th=threshold_otsu(a)
b=a<th
plt.subplot(1,2,2)
plt.imshow(a<th)

In [ ]:
distance = ndi.distance_transform_edt(b)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((9,9)),labels=b)
markers = ndi.label(local_maxi)[0]

labels = watershed(-distance, markers, mask=b)

In [ ]:
cmap = colors.ListedColormap ( np.random.rand ( 256,3))
plt.figure(figsize=[15,4])
plt.imshow(np.transpose(labels),cmap=cmap)

In [ ]:
h,ax=np.histogram(labels,bins=np.arange(0,labels.max()+1))
plt.plot(h[1:])
plt.plot([0,47],[h[1:].mean(), h[1:].mean()],'g')
plt.plot([0,47],[h[1:].mean()-h[1:].std(), h[1:].mean()-h[1:].std()],'r')
plt.plot([0,47],[h[1:].mean()+h[1:].std(), h[1:].mean()+h[1:].std()],'r')

In [ ]:
m=h[np.argwhere(h<0.1*np.prod(labels.shape))].mean()
s=h[1:].std()
print(m,s,m-s,m+s)


In [ ]:
def findBeads(img) :
    distance = ndi.distance_transform_edt(img)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((9,9)),labels=b)
    markers = ndi.label(local_maxi)[0]

    labels = watershed(-distance, markers, mask=b)
    h,ax=np.histogram(labels,bins=np.arange(0,labels.max()+1))
    m=h[np.argwhere(h<0.05*np.prod(labels.shape))].mean()
    s=h[np.argwhere(h<0.05*np.prod(labels.shape))].std()
    cog=[]
    for i in np.arange(0, h.size) :
        if ((m-s)<h[i]) and (h[i]<(m+s)) :
            selection=np.argwhere(labels==i)
            cog.append(selection.mean(axis=0))
       
    cog=np.asarray(cog)
    return cog


In [ ]:
cog=findBeads(b)
print(cog) 
plt.figure(figsize=[15,4])
plt.imshow(b)

plt.plot(cog[:,1]-1,cog[:,0]-1,'r+')

In [ ]:
labels.shape

In [ ]:
pos.mean(axis=0)